In [1]:
import sys
import re
from glob import glob
import os
from itertools import groupby
import pickle
import bs4
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import json

import gensim.downloader as api
model = api.load('word2vec-google-news-300')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

unable to import 'smart_open.gcs', disabling that module


In [26]:
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [14]:
print(model.distance('sweet', 'sugar'))    
print(model.distance('sweet', 'honey'))   
print(model.distance('sweet', 'chicken'))
print(model.distance('sweet', 'salt'))
print(model.distance('salty', 'chicken')) 
print(model.distance('salty', 'honey'))
print(model.distance('salty', 'sweet'))
print(model.distance('salt', 'sweet'))


0.7021858096122742
0.5962503254413605
0.8297711908817291
0.8297542631626129
0.7185156345367432
0.660496860742569
0.5050951242446899
0.8297542631626129


In [ ]:
# recipes={}
# inv_idx={}
# ing_list=[]
# with open('epi_r.csv', newline='',encoding='utf-8') as csvfile:
#     csvreader = csv.reader(csvfile, delimiter=',', quotechar='|')
#     for row in csvreader:
#         if not inv_idx:
#             ing_list=row
#             for idx in range(1,len(row)):
#                 inv_idx[row[idx]]=[]
#         else:
#             recipe=row[0].rstrip()
#             recipes[recipe]={}
#             recipes[recipe]['nutrition']={}
#             recipes[recipe]['features']={}
#             for idx in range(2,len(row)):
#                 recipe_name=row[0]
#                 if row[idx]!='':
#                     if int(row[idx])>0:
#                         #add recipe name to inv_idx
#                         if idx>5:
#                             inv_idx[ing_list[idx]].append(recipe_name.rstrip())
#                             #add ingredients recipe dictionary
#                             recipes[recipe]['features'][ing_list[idx]]=int(row[idx])
#                         else: 
#                             recipes[recipe]['nutrition'][ing_list[idx]]=int(row[idx])


    #print(recipes.keys())
    # print('Lentil Apple and Turkey Wrap',": ",recipes['Lentil Apple and Turkey Wrap'])
            
    # print(inv_idx['apple'])

In [51]:
class IngredientIndex:
    ''' Class representing an inverted index search tool'''
    
    def __init__(self):
        
        self.recipes={}
        self.inv_idx={}
    
        with open('full_format_recipes.json', newline='', encoding='utf-8') as f:
            recipes_list = json.load(f)
            
            for recipe_dict in recipes_list:
                if 'title' in recipe_dict:
                    title = recipe_dict['title'].rstrip().replace(',','')
                    self.recipes[title] = {}
                    self.recipes[title]['features'] = set()
                    self.recipes[title]['rating'] = recipe_dict['rating']
                    self.recipes[title]['directions'] = recipe_dict['directions']
                    self.recipes[title]['calories'] = recipe_dict['calories']
                    self.recipes[title]['ingredients'] = recipe_dict['ingredients']
                    self.recipes[title]['desc'] = recipe_dict['desc']
                    
                    if self.recipes[title]['desc'] is None:
                        self.recipes[title]['desc'] = 'No description available.'
                    
                    for ingredient_str in recipe_dict['ingredients']:
                        ingredients = ingredient_str.split()
                        # pos = nltk.pos_tag(ingredients)
                        for ingredient in ingredients:
                            # if tag in {'NN', 'NNS', 'NNP', 'NNPS'}:
                            ingredient = lemmatizer.lemmatize(ingredient.lower().replace(',',''))
                            self.recipes[title]['features'].add(ingredient)
                            
                    for ingredient in title.split():
                            ingredient = lemmatizer.lemmatize(ingredient.lower().replace(',',''))
                            self.recipes[title]['features'].add(ingredient)
                        
                    for ingredient in recipe_dict['categories']:
                            ingredient = lemmatizer.lemmatize(ingredient.lower().replace(',',''))
                            self.recipes[title]['features'].add(ingredient)
                    
                    for ingredient in self.recipes[title]['features']:
                        if ingredient not in self.inv_idx:
                            self.inv_idx[ingredient] = set()
                        self.inv_idx[ingredient].add(title)
                        
    def __getitem__(self, item):
        return self.inv_idx[lemmatizer.lemmatize(item)]
    
    # Returns set of recipes such that every ingredient in ingredient_list is included in each recipe
    def intersect(self, ingredient_list):
        
        result = -1
        
        for ingredient in ingredient_list:
            if result == -1:
                result = self[ingredient]
            else:
                result = result.intersection(self[ingredient])
        
        if result == -1:
            result = set()
        
        return result

    # Returns set of recipes such that at least one ingredient in ingredient_list is included in each recipe
    def union(self, ingredient_list):
        
        result = set()
        
        for ingredient in ingredient_list:
            result = result.union(self[ingredient])
                
        return result
    
    # Returns set of recipes such that no ingredient in ingredient_list is included in each recipe
    def exclude(self, ingredient_list):
        
        bad_recipes = self.union(ingredient_list)
        
        result = set()
        
        for recipe in self.recipes:
            if recipe not in bad_recipes:
                result.add(recipe)
        
        return result
    
    # Returns set of recipes such that every ingredient in good_ingredient_list is included in each recipe
    # and every ingredient in bad_ingredient_list is not included in each recipe
    def search(self, good_ingredient_list, bad_ingredient_list):
        
        return self.intersect(good_ingredient_list).intersection(self.exclude(bad_ingredient_list))
    
temp = IngredientIndex()

In [31]:
# print(temp['french'])

print(temp.recipes['Romesco Sauce'])

print(len(temp['apples']))

print(len(temp.exclude([])))

print(len(temp.intersect(['apple', 'chicken'])))

# print('Cider-Braised Chicken' in temp['lemon'])

print(len(temp.exclude(['lemon', 'apple', 'chicken'])))

print(temp.search(['tree nut free'], ['apricot']))

# No way to sort by flavor (i.e. salty, sweet)
# No way to filter by allergies or dietary restrictions that encompass more than a single ingredient
# Doesn't correctly deal with ingredients with multi-word names (i.e. black pepper)

{'features': {'parsley', 'labeled', 'bell', 'pepper', '1/2', 'sherry', 'red', '1/4', 'slivered', 'clove', 'oil', 'smoked', 'olive', 'bon appétit', 'tablespoon', 'pimentón', 'fine', 'smashed', 'quick & easy', 'paprika', 'at', 'freshly', 'vinegar', 'sometimes', 'from', 'black', 'low cal', 'teaspoon', 'spanish', 'ground', 'info:', 'supermarkets.', '1', 'roasted', 'salt', 'vera', 'extra-virgin', 'latienda.com.', 'large', 'low cholesterol', 'vegetarian', 'flat-leaf', 'almond', 'toasted', 'available', 'vegan', 'specialty', 'the', 'sauce', 'a', 'and', 'sold', 'store', 'romesco', 'purée', 'la', 'tomato', 'sea', 'most', 'cayenne', 'or', 'de', 'dulce', '2', 'food', 'garlic', 'ingredient', 'cup', 'chopped', 'type', 'is', 'jar'}, 'rating': 5.0, 'directions': ['Pulse first 8 ingredients in a food processor until very finely chopped. With motor running, slowly add oil; process until smooth. Season with salt and pepper. DO AHEAD: Romesco can be made 1 week ahead. Cover and chill.'], 'calories': 430.0

In [43]:
sweet_dict = {}
print(nltk.pos_tag(['chicken']))
for ingredient in temp.inv_idx:
    try:
        a = nltk.pos_tag([ingredient])
        if a[0][1] in {'NN', 'NNS', 'NNP', 'NNPS'}:
            try:
                sweet_dict[ingredient] = model.distance('sweet', ingredient)
            except KeyError:
                pass
    except IndexError:
        print(ingredient)


res = sorted(sweet_dict.items(), key=lambda t: t[1])
print(res[:50])
print()
print(res[-50:])


[('chicken', 'NN')]

[('sweet', 0.0), ('sweetness', 0.378318727016449), ('yummy', 0.4036175608634949), ('buttery', 0.4060057997703552), ('fruity', 0.41959458589553833), ('peanutty', 0.4317047595977783), ('chocolaty', 0.4353603720664978), ('tangy', 0.4422646760940552), ('sweeter', 0.4452689290046692), ('tart', 0.4511781334877014), ('creamy', 0.4532545804977417), ('lemony', 0.45816218852996826), ('savory', 0.45902931690216064), ('bacony', 0.4617229104042053), ('spicy', 0.4639192819595337), ('gooey', 0.4700455665588379), ('crunchy', 0.47281336784362793), ('peppery', 0.4830467700958252), ('bittersweet', 0.48885995149612427), ('zesty', 0.49116939306259155), ('caramel', 0.49504488706588745), ('jammy', 0.5015853643417358), ('salty', 0.5050951242446899), ('fragrant', 0.5056147873401642), ('fudgy', 0.5076992213726044), ('juicy', 0.5080854594707489), ('citrusy', 0.5081562697887421), ('oaty', 0.5107993185520172), ('gingery', 0.5110805332660675), ('chewy', 0.5142950415611267), ('clafoutis', 0.5150

In [45]:
print(nltk.pos_tag(['spicy']))

[('spicy', 'NN')]


In [ ]:
for item in inv_idx['apple']:
    if item not in temp['apple']:
        print(item)

In [ ]:
for item in temp['apple']:
    print(item)
#     if item not in inv_idx['apple']:
#         print(item)